In [ ]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, log, exp
from biogeme.models import loglogit, piecewise_formula, boxcox, lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from scipy.stats import chi2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
# Loading the data
df = pd.read_csv('lpmc06.dat', sep='\t')

In [ ]:
df["dur_pt"] = df["dur_pt_access"] + df["dur_pt_rail"] + df["dur_pt_bus"] + df["dur_pt_int"]
df["cost_driving"] = df["cost_driving_fuel"] + df["cost_driving_ccharge"]

In [ ]:
df["seg_age"] = df["age"].apply(lambda x: 0 if x < 35 else 1)

In [ ]:
display(df.columns)

In [ ]:
display(df.head())

In [ ]:
database = db.Database('lpmc', df)

In [ ]:
df[["dur_walking", "dur_cycling", "dur_pt", "dur_driving"]].describe()

## Model 0

We identify the variables that will enter the model specification.

In [ ]:
# Time related variables
dur_walking = Variable('dur_walking') # in hours
dur_cycling = Variable('dur_cycling') # in hours
# dur_pt_access = Variable('dur_pt_access') # in hours
# dur_pt_rail = Variable('dur_pt_rail') # in hours
# dur_pt_bus = Variable('dur_pt_bus') # in hours
# dur_pt_int = Variable('dur_pt_int') # in hours
dur_pt = Variable('dur_pt')
dur_driving = Variable('dur_driving') # in hours


# Cost related variables
cost_transit = Variable('cost_transit') # in GBP
# cost_driving_fuel = Variable('cost_driving_fuel') # in GBP
# cost_driving_ccharge = Variable('cost_driving_ccharge') # in GBP
cost_driving = Variable('cost_driving')

# Choice taken by the individual
travel_mode = Variable('travel_mode') # 1 = walk, 2 = cycle, 3 = PT, 4 = car

Parameters to be estimated

In [ ]:
# ASC_WALK = Beta('asc_walk', 0, None, None, 0)
ASC_CYCLE = Beta('asc_cycle', 0, None, None, 0)
ASC_PT = Beta('asc_pt', 0, None, None, 0)
ASC_CAR = Beta('asc_car', 0, None, None, 0)

B_TIME = Beta('b_time', 0, None, None, 0)
B_COST = Beta('b_cost', 0, None, None, 0)

Definition of the utility functions.

In [ ]:
# Walk
V1 = (
  # ASC_WALK -> Normalized with respect to walk
  B_TIME * dur_walking
)

# Cycle
V2 = (
  ASC_CYCLE
  + B_TIME * dur_cycling
)

# Public transport
V3 = (
  ASC_PT
  + B_TIME * dur_pt
  #+ B_TIME * (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int)
  + B_COST * cost_transit
)

# Car
V4 = (
  ASC_CAR
  + B_TIME * dur_driving
  + B_COST * cost_driving
  #+ B_COST * (cost_driving_fuel + cost_driving_ccharge)
)

In [ ]:
V = {1: V1, 2: V2, 3: V3, 4: V4}

Definition of the model.

In [ ]:
# All alternatives are available to all individuals.
logprob = loglogit(V, None, travel_mode)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_0'

Estimate the parameters.

In [ ]:
results = biogeme.estimate()

Summary statistics.

In [ ]:
print(results.print_general_statistics())

In [ ]:
display(results.get_estimated_parameters())

## Model 1
In this version of the model, we introduce alternative-specific coefficients for either b_time (model 1a) or b_cost (model 1b), and we compare the results to determine which model yields the best results 

### Model 1a (alternative-specific b_time)


Parameters to be estimated

In [ ]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_1A = Beta('ASC_CYCLE_1A', 0, None, None, 0)
ASC_PT_1A    = Beta('ASC_PT_1A', 0, None, None, 0)
ASC_CAR_1A   = Beta('ASC_CAR_1A', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_1A  = Beta('B_TIME_WALK_1A', 0, None, None, 0)
B_TIME_CYCLE_1A = Beta('B_TIME_CYCLE_1A', 0, None, None, 0)
B_TIME_PT_1A    = Beta('B_TIME_PT_1A', 0, None, None, 0)
B_TIME_CAR_1A   = Beta('B_TIME_CAR_1A', 0, None, None, 0)

# Generic cost coefficient
B_COST_1A = Beta('B_COST_1A', 0, None, None, 0)

Definition of the utility functions

In [ ]:
V_walk_1A = (
    B_TIME_WALK_1A * dur_walking
)

V_cycle_1A = (
    ASC_CYCLE_1A
    + B_TIME_CYCLE_1A * dur_cycling
)

V_PT_1A = (
    ASC_PT_1A
    + B_TIME_PT_1A * dur_pt
    + B_COST_1A * cost_transit
)

V_car_1A = (
    ASC_CAR_1A
    + B_TIME_CAR_1A * dur_driving
    + B_COST_1A * cost_driving
)

V_1A = {1: V_walk_1A, 2: V_cycle_1A, 3: V_PT_1A, 4: V_car_1A}

Definition of the model

In [ ]:
logprob_1A = loglogit(V_1A, None, travel_mode)
biogeme_1A = bio.BIOGEME(database, logprob_1A)
biogeme_1A.modelName = 'model_1A'

Estimation and display of the results

In [ ]:
results_1A = biogeme_1A.estimate()
print(results_1A.print_general_statistics())

In [ ]:
display(results_1A.get_estimated_parameters())

_Observations for the report_

First, we observe that all the B coefficients are negative, which makes sense, because a longer and/or more expensive travel mode is less attractive. 

The cost coefficient of model 1A is very close to the one for the model 0. On the other hand, the time coefficient, which was made alternative specific in model 1A, now strongly depends on the chosen mode. In particular, we observe it is the smallest (in absolute value) for PT; an interpretation could be that commuters are more prone to long PT travel times because they can read, sleep, etc., activites they cannot do while driving or cycling. On the other hand, the largest time_coefficient (again, in absolute value) is for walking, probably because beyond 20-30 minutes, people consider that doing the route by foot is too long.

If we compare the Akaike or Bayesian information criterion between model 0 and model 1a, we observe they are both lower in the case of model 1a; it means the latter fits the data better than model 0.

### Model 1b (alternative-specific b_cost)

We reproduce exactly the same steps as for model 1A, but we now assume a generic time coefficient, and a alternative specific cost coefficient

Parameters to be estimated

In [ ]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_1B = Beta('ASC_CYCLE_1B', 0, None, None, 0)
ASC_PT_1B    = Beta('ASC_PT_1B', 0, None, None, 0)
ASC_CAR_1B   = Beta('ASC_CAR_1B', 0, None, None, 0)

# Alternative specific cost coefficient
B_COST_PT_1B    = Beta('B_COST_PT_1B', 0, None, None, 0)
B_COST_CAR_1B   = Beta('B_COST_CAR_1B', 0, None, None, 0)

# Generic time coefficient
B_TIME_1B = Beta('B_TIME_1B', 0, None, None, 0)

Definition of the utility functions 

In [ ]:
V_walk_1B = (
    B_TIME_1B * dur_walking
)

V_cycle_1B = (
    ASC_CYCLE_1B
    + B_TIME_1B * dur_cycling
)

V_PT_1B = (
    ASC_PT_1B
    + B_TIME_1B * dur_pt
    + B_COST_PT_1B * cost_transit
)

V_car_1B = (
    ASC_CAR_1B
    + B_TIME_1B * dur_driving
    + B_COST_CAR_1B * cost_driving
)

V_1B = {1: V_walk_1B, 2: V_cycle_1B, 3: V_PT_1B, 4: V_car_1B}

Definition of the model

In [ ]:
logprob_1B = loglogit(V_1B, None, travel_mode)
biogeme_1B = bio.BIOGEME(database, logprob_1B)
biogeme_1B.modelName = 'model_1B'

Estimation and display of the results

In [ ]:
results_1B = biogeme_1B.estimate()
print(results_1B.print_general_statistics())

In [ ]:
display(results_1B.get_estimated_parameters())

In [ ]:
lr_result1A = results_1A.likelihood_ratio_test(results, 0.05)
print(f'{lr_result1A.statistic=:.3g}')
print(f'{lr_result1A.threshold=:.3g}')
print(lr_result1A.message)

In [ ]:
lr_result1B = results_1B.likelihood_ratio_test(results, 0.05)
print(f'{lr_result1B.statistic=:.3g}')
print(f'{lr_result1B.threshold=:.3g}')
print(lr_result1B.message)

_Observations for the report_

This model seems clearly less effective than model 1A. We see it because

1) The Bayesian and Akaike criterion are higher for model 1B
2) the t-test are closer to 0
3) the cost coefficient for PT is higher than 0, which doesn't make much sense

We could have guessed that model 1B would be less interesting than model 1A, because only two alternatives out of four have a cost parameter, which means that making B_cost alternative specific allows less flexibility in the model than making B_time alternative specific

So our preferred model for the rest of the project will be model 1A (alternative specific time coefficients)

To formalize this, we perform a Cox test. We define a general model containing both an alternative-specific b_time and alt-spec b_cost

In [ ]:
# Utility functions
V_walk_1Cox = (B_TIME_WALK_1A * dur_walking)

V_cycle_1Cox = (
    ASC_CYCLE
    + B_TIME_CYCLE_1A * dur_cycling
)

V_PT_1Cox = (
    ASC_PT
    + B_TIME_PT_1A * dur_pt
    + B_COST_PT_1B * cost_transit
)

V_car_1Cox = (
    ASC_CAR
    + B_TIME_CAR_1A * dur_driving
    + B_COST_CAR_1B * cost_driving
)

V_1Cox = {1: V_walk_1Cox, 2: V_cycle_1Cox, 3: V_PT_1Cox, 4: V_car_1Cox}

# Estimation of the results
logprob_1Cox = loglogit(V_1Cox, None, travel_mode)
biogeme_1Cox = bio.BIOGEME(database, logprob_1Cox)
biogeme_1Cox.modelName = 'model_1Cox'
results_1Cox = biogeme_1Cox.estimate()

We compare model 1C with model 1A

In [ ]:
lr_result1AC = results_1A.likelihood_ratio_test(results_1Cox, 0.005)
print(f'{lr_result1AC.statistic=:.3g}')
print(f'{lr_result1AC.threshold=:.3g}')
print(lr_result1AC.message)

We compare model 1C with model 1B

In [ ]:
lr_result1BC = results_1B.likelihood_ratio_test(results_1Cox, 0.005)
print(f'{lr_result1BC.statistic=:.3g}')
print(f'{lr_result1BC.threshold=:.3g}')
print(lr_result1BC.message)

In [ ]:
V_pref = V_1A
results_pref = results_1A

To conclude, model 1A is better than model 1B because, at the level $\alpha = 0.5 \%$, model B can be rejected against the composite model, but model A cannot. By the Cox test, 1A is better than 1B

## Model 2

#### Age

In [ ]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_2A = Beta('ASC_CYCLE_2A', 0, None, None, 0)
ASC_PT_2A    = Beta('ASC_PT_2A', 0, None, None, 0)
ASC_CAR_2A   = Beta('ASC_CAR_2A', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_2A  = Beta('B_TIME_WALK_2A', 0, None, None, 0)
B_TIME_CYCLE_2A = Beta('B_TIME_CYCLE_2A', 0, None, None, 0)
B_TIME_PT_2A    = Beta('B_TIME_PT_2A', 0, None, None, 0)
B_TIME_CAR_2A   = Beta('B_TIME_CAR_2A', 0, None, None, 0)

# Generic cost coefficient
B_COST_2A = Beta('B_COST_2A', 0, None, None, 0)

age = Variable('age')
B_AGE_CYCLE_2A = Beta('B_AGE_CYCLE_2A', 0, None, None, 0)
B_AGE_PT_2A = Beta('B_AGE_PT_2A', 0, None, None, 0)
B_AGE_CAR_2A = Beta('B_AGE_CAR_2A', 0, None, None, 0)

In [ ]:
V_walk_2A = (
    B_TIME_WALK_2A * dur_walking
)

V_cycle_2A = (
    ASC_CYCLE_2A
    + B_AGE_CYCLE_2A * age
    + B_TIME_CYCLE_2A * dur_cycling
)

V_pt_2A = (
    ASC_PT_2A
    + B_AGE_PT_2A * age
    + B_TIME_PT_2A * dur_pt
    + B_COST_2A * cost_transit
)

V_car_2A = (
    ASC_CAR_2A
    + B_AGE_CAR_2A * age
    + B_TIME_CAR_2A * dur_driving
    + B_COST_2A * cost_driving
)

V_2A ={1: V_walk_2A, 2: V_cycle_2A, 3: V_pt_2A, 4: V_car_2A}

logprob_2A = loglogit(V_2A, None, travel_mode)
biogeme_2A = bio.BIOGEME(database, logprob_2A)
biogeme_2A.modelName = 'model_2A'

results_2A = biogeme_2A.estimate()
print(results_2A.print_general_statistics())

In [ ]:
display(results_2A.get_estimated_parameters())

In [ ]:
lr_result2A = results_2A.likelihood_ratio_test(results_pref, 0.05)
print(f'{lr_result2A.statistic=:.3g}')
print(f'{lr_result2A.threshold=:.3g}')
print(lr_result2A.message)

In [ ]:
V_walk_2A_significant = (
    B_TIME_WALK_2A * dur_walking
)

V_cycle_2A_significant = (
    ASC_CYCLE_2A
    # + B_AGE_CYCLE_2A * age
    + B_TIME_CYCLE_2A * dur_cycling
)

V_pt_2A_significant = (
    ASC_PT_2A
    + B_AGE_PT_2A * age
    + B_TIME_PT_2A * dur_pt
    + B_COST_2A * cost_transit
)

V_car_2A_significant = (
    ASC_CAR_2A
    + B_AGE_CAR_2A * age
    + B_TIME_CAR_2A * dur_driving
    + B_COST_2A * cost_driving
)

V_2A_significant ={1: V_walk_2A_significant, 2: V_cycle_2A_significant, 3: V_pt_2A_significant, 4: V_car_2A_significant}

logprob_2A_significant = loglogit(V_2A_significant, None, travel_mode)
biogeme_2A_significant = bio.BIOGEME(database, logprob_2A_significant)
biogeme_2A_significant.modelName = 'model_2A_significant'

results_2A_significant = biogeme_2A_significant.estimate()
print(results_2A_significant.print_general_statistics())

In [ ]:
display(results_2A_significant.get_estimated_parameters())

In [ ]:
lr_result2A_significant = results_2A_significant.likelihood_ratio_test(results_pref, 0.05)
print(f'{lr_result2A_significant.statistic=:.3g}')
print(f'{lr_result2A_significant.threshold=:.3g}')
print(lr_result2A_significant.message)

In [ ]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_2B = Beta('ASC_CYCLE_2B', 0, None, None, 0)
ASC_PT_2B    = Beta('ASC_PT_2B', 0, None, None, 0)
ASC_CAR_2B   = Beta('ASC_CAR_2B', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_2B  = Beta('B_TIME_WALK_2B', 0, None, None, 0)
B_TIME_CYCLE_2B = Beta('B_TIME_CYCLE_2B', 0, None, None, 0)
B_TIME_PT_2B    = Beta('B_TIME_PT_2B', 0, None, None, 0)
B_TIME_CAR_2B   = Beta('B_TIME_CAR_2B', 0, None, None, 0)

# Generic cost coefficient
B_COST_2B = Beta('B_COST_2B', 0, None, None, 0)

age = Variable('age')
B_AGE_CYCLE_2B = Beta('B_AGE_CYCLE_2B', 0, None, None, 0)
B_AGE_PT_2B = Beta('B_AGE_PT_2B', 0, None, None, 0)
B_AGE_CAR_2B = Beta('B_AGE_CAR_2B', 0, None, None, 0)

In [ ]:
V_walk_2B = (
    B_TIME_WALK_2B * dur_walking 
)

V_cycle_2B = (
    ASC_CYCLE_2B
    + B_TIME_CYCLE_2B * dur_cycling 
)

V_PT_2B = (
    ASC_PT_2B
    + B_TIME_PT_2B * dur_pt 
    + B_COST_2B * cost_transit * age
)

V_car_2B = (
    ASC_CAR_2B
    + B_TIME_CAR_2B * dur_driving 
    + B_COST_2B * cost_driving * age
)

V_2B ={1: V_walk_2B, 2: V_cycle_2B, 3: V_PT_2B, 4: V_car_2B}

logprob_2B = loglogit(V_2B, None, travel_mode)
biogeme_2B = bio.BIOGEME(database, logprob_2B)
biogeme_2B.modelName = 'model_2B'

results_2B = biogeme_2B.estimate()
print(results_2B.print_general_statistics())

In [ ]:
display(results_2B.get_estimated_parameters())

In [ ]:
lr_result2B = results_2B.likelihood_ratio_test(results_pref, 0.05)
print(f'{lr_result2B.statistic=:.3g}')
print(f'{lr_result2B.threshold=:.3g}')
print(lr_result2B.message)

In [ ]:
V_pref = V_2A_significant
results_pref = results_2A_significant

## Model 3

Using Model_pref as the base model, include an appropriate non-linear transformation of one of the variables. Report both the specication and the estimation results (as defined previously).

In [ ]:
sns.histplot(df["dur_walking"])
sns.histplot(df["dur_cycling"])
sns.histplot(df["dur_pt_access"] + df["dur_pt_rail"] + df["dur_pt_bus"] + df["dur_pt_int"])
sns.histplot(df["dur_driving"])
# plt.xscale('log')
plt.legend(['Walking', 'Cycling', 'Public transport', 'Driving'])
plt.show()

#### Piecewise Linear

In [ ]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_3A = Beta('ASC_CYCLE_3A', 0, None, None, 0)
ASC_PT_3A    = Beta('ASC_PT_3A', 0, None, None, 0)
ASC_CAR_3A   = Beta('ASC_CAR_3A', 0, None, None, 0)

# Generic cost coefficient
B_COST_3A = Beta('B_COST_3A', 0, None, None, 0)

age = Variable('age')
# B_AGE_CYCLE_3A = Beta('B_AGE_CYCLE_3A', 0, None, None, 0)
B_AGE_PT_3A = Beta('B_AGE_PT_3A', 0, None, None, 0)
B_AGE_CAR_3A = Beta('B_AGE_CAR_3A', 0, None, None, 0)

In [ ]:
thresholds = [0, 0.1, 0.5, 1, 2, None]

BETA_TIME_3A_1 = Beta('BETA_TIME_3A_0_0.1', 0, None, None, 0)
BETA_TIME_3A_2 = Beta('BETA_TIME_3A_0.1_0.5', 0, None, None, 0)
BETA_TIME_3A_3 = Beta('BETA_TIME_3A_0.5_1', 0, None, None, 0)
BETA_TIME_3A_4 = Beta('BETA_TIME_3A_1_2', 0, None, None, 0)
BETA_TIME_3A_5 = Beta('BETA_TIME_3A_2_more', 0, None, None, 0)
betas_piecewise = [BETA_TIME_3A_1, BETA_TIME_3A_2, BETA_TIME_3A_3, BETA_TIME_3A_4, BETA_TIME_3A_5]

In [ ]:
piecewise_dur_walking = piecewise_formula(
    dur_walking, thresholds, betas_piecewise
)
piecewise_dur_cycling = piecewise_formula(
    dur_cycling, thresholds, betas_piecewise
)
piecewise_dur_pt = piecewise_formula(
    dur_pt, thresholds, betas_piecewise
)
piecewise_dur_driving = piecewise_formula(
    dur_driving, thresholds, betas_piecewise
)

In [ ]:
V_walk_3A_piecewise = (
    piecewise_dur_walking
)

V_cycle_3A_piecewise = (
    ASC_CYCLE_3A
    # + B_AGE_CYCLE_3A * age
    + piecewise_dur_cycling
)

V_pt_3A_piecewise = (
    ASC_PT_3A
    + B_AGE_PT_3A * age
    + piecewise_dur_pt
    + B_COST_3A * cost_transit
)

V_car_3A_piecewise = (
    ASC_CAR_3A
    + B_AGE_CAR_3A * age
    + piecewise_dur_driving
    + B_COST_3A * cost_driving
)

V_3A_piecewise ={1: V_walk_3A_piecewise, 2: V_cycle_3A_piecewise, 3: V_pt_3A_piecewise, 4: V_car_3A_piecewise}
logprob_3A_piecewise = loglogit(V_3A_piecewise, None, travel_mode)
biogeme_3A_piecewise = bio.BIOGEME(database, logprob_3A_piecewise)
biogeme_3A_piecewise.modelName = 'model_3A_piecewise'

results_3A_piecewise = biogeme_3A_piecewise.estimate()
print(results_3A_piecewise.print_general_statistics())

In [ ]:
results_3A_piecewise.getEstimatedParameters()

In [ ]:
piecewise_estimates = results_3A_piecewise.get_beta_values()
V_walk_3A_piecewise.fix_betas(beta_values=piecewise_estimates, prefix='piecewise_')
V_cycle_3A_piecewise.fix_betas(beta_values=piecewise_estimates, prefix='piecewise_')
V_pt_3A_piecewise.fix_betas(beta_values=piecewise_estimates, prefix='piecewise_')
V_car_3A_piecewise.fix_betas(beta_values=piecewise_estimates, prefix='piecewise_')

In [ ]:
alpha = Beta('alpha', 0, None, None, 0)
Opt1 = (1 - alpha) * V_walk_2A + alpha * V_walk_3A_piecewise
Opt2 = (1 - alpha) * V_cycle_2A + alpha * V_cycle_3A_piecewise
Opt3 = (1 - alpha) * V_pt_2A + alpha * V_pt_3A_piecewise
Opt4 = (1 - alpha) * V_car_2A + alpha * V_car_3A_piecewise

V = {1: Opt1, 2: Opt2, 3: Opt3, 4: Opt4}
logprob = loglogit(V, None, travel_mode)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_3A_composite'
results_composite = biogeme.estimate()


In [ ]:
composite_parameters = results_composite.get_estimated_parameters()
display(composite_parameters)

In [ ]:
t_test_alpha = composite_parameters.loc['alpha', 'Rob. t-test']
print(f't-test for alpha: {t_test_alpha:.3g}')

In [ ]:
p_value_alpha = composite_parameters.loc['alpha', 'Rob. p-value']
print(f'p-value for alpha: {p_value_alpha:.3g}')


Under the null hypothesis, the true value of $\alpha$ is 0. This hypothesis can be safely rejected here. Therefore, the piecewise linear specification is preferred.

#### Box-Cox

In [ ]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_3B = Beta('ASC_CYCLE_3B', 0, None, None, 0)
ASC_PT_3B    = Beta('ASC_PT_3B', 0, None, None, 0)
ASC_CAR_3B   = Beta('ASC_CAR_3B', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_3B  = Beta('B_TIME_WALK_3B', 0, None, None, 0)
B_TIME_CYCLE_3B = Beta('B_TIME_CYCLE_3B', 0, None, None, 0)
B_TIME_PT_3B    = Beta('B_TIME_PT_3B', 0, None, None, 0)
B_TIME_CAR_3B   = Beta('B_TIME_CAR_3B', 0, None, None, 0)

# Generic cost coefficient
B_COST_3B = Beta('B_COST_3B', 0, None, None, 0)

age = Variable('age')
# B_AGE_CYCLE_3B = Beta('B_AGE_CYCLE_3B', 0, None, None, 0)
B_AGE_PT_3B = Beta('B_AGE_PT_3B', 0, None, None, 0)
B_AGE_CAR_3B = Beta('B_AGE_CAR_3B', 0, None, None, 0)

In [ ]:
lambda_boxcox_walking = Beta('lambda_boxcox_walking', 1, -10, 10, 0)
boxcox_dur_walking_unrestricted = boxcox(dur_walking, lambda_boxcox_walking)

lambda_boxcox_cycling = Beta('lambda_boxcox_cycling', 1, -10, 10, 0)
boxcox_dur_cycling_unrestricted = boxcox(dur_cycling, lambda_boxcox_cycling)

lambda_boxcox_pt = Beta('lambda_boxcox_pt', 1, -10, 10, 0)
boxcox_dur_pt_unrestricted = boxcox(dur_pt, lambda_boxcox_pt)

lambda_boxcox_driving = Beta('lambda_boxcox_driving', 1, -10, 10, 0)
boxcox_dur_driving_unrestricted = boxcox(dur_driving, lambda_boxcox_driving)

In [ ]:
V_walk_3B_unrestricted = (
    B_TIME_WALK_3B * boxcox_dur_walking_unrestricted
)

V_cycle_3B_unrestricted = (
    ASC_CYCLE_3B
    # + B_AGE_CYCLE_3B * age
    + B_TIME_CYCLE_3B * boxcox_dur_cycling_unrestricted
)

V_pt_3B_unrestricted = (
    ASC_PT_3B
    + B_AGE_PT_3B * age
    + B_TIME_PT_3B * boxcox_dur_pt_unrestricted
    + B_COST_3B * cost_transit
)

V_car_3B_unrestricted = (
    ASC_CAR_3B
    + B_AGE_CAR_3B * age
    + B_TIME_CAR_3B * boxcox_dur_driving_unrestricted
    + B_COST_3B * cost_driving
)

V_3B_unrestricted ={1: V_walk_3B_unrestricted, 2: V_cycle_3B_unrestricted, 3: V_pt_3B_unrestricted, 4: V_car_3B_unrestricted}
logprob_3B_unrestricted = loglogit(V_3B_unrestricted, None, travel_mode)
biogeme_3B_unrestricted = bio.BIOGEME(database, logprob_3B_unrestricted)
biogeme_3B_unrestricted.modelName = 'model_3B_unrestricted'

results_3B_unrestricted = biogeme_3B_unrestricted.estimate()
print(results_3B_unrestricted.print_general_statistics())

In [ ]:
results_3B_unrestricted.getEstimatedParameters()

In [ ]:
lr_result3B = results_3B_unrestricted.likelihood_ratio_test(results_pref, 0.05)
print(f'{lr_result3B.statistic=:.3g}')
print(f'{lr_result3B.threshold=:.3g}')
print(lr_result3B.message)

In [ ]:
lambda_boxcox = Beta('lambda_boxcox', 1, -10, 10, 0)
boxcox_dur_walking_restricted = boxcox(dur_walking, lambda_boxcox)
boxcox_dur_cycling_restricted = boxcox(dur_cycling, lambda_boxcox)
boxcox_dur_pt_restricted = boxcox(dur_pt, lambda_boxcox)
boxcox_dur_driving_restricted = boxcox(dur_driving, lambda_boxcox)

In [ ]:
V_walk_3B_restricted = (
    B_TIME_WALK_3B * boxcox_dur_walking_restricted
)

V_cycle_3B_restricted = (
    ASC_CYCLE_3B
    # + B_AGE_CYCLE_3B * age
    + B_TIME_CYCLE_3B * boxcox_dur_cycling_restricted
)

V_pt_3B_restricted = (
    ASC_PT_3B
    + B_AGE_PT_3B * age
    + B_TIME_PT_3B * boxcox_dur_pt_restricted
    + B_COST_3B * cost_transit
)

V_car_3B_restricted = (
    ASC_CAR_3B
    + B_AGE_CAR_3B * age
    + B_TIME_CAR_3B * boxcox_dur_driving_restricted
    + B_COST_3B * cost_driving
)

V_3B_restricted = {1: V_walk_3B_restricted, 2: V_cycle_3B_restricted, 3: V_pt_3B_restricted, 4: V_car_3B_restricted}
logprob_3B_restricted = loglogit(V_3B_restricted, None, travel_mode)
biogeme_3B_restricted = bio.BIOGEME(database, logprob_3B_restricted)
biogeme_3B_restricted.modelName = 'model_3B_restricted'

results_3B_restricted = biogeme_3B_restricted.estimate()
print(results_3B_restricted.print_general_statistics())

In [ ]:
results_3B_restricted.getEstimatedParameters()

In [ ]:
# lr_result3B = results_3B_restricted.likelihood_ratio_test(results_pref, 0.05)
# print(f'{lr_result3B.statistic=:.3g}')
# print(f'{lr_result3B.threshold=:.3g}')
# print(lr_result3B.message)

Testing the unrestricted model where each utility has its own lambda, with the restriction of all lambdas equal.

In [ ]:
lr_result3B = results_3B_unrestricted.likelihood_ratio_test(results_3B_restricted, 0.05)
print(f'{lr_result3B.statistic=:.3g}')
print(f'{lr_result3B.threshold=:.3g}')
print(lr_result3B.message)

#### Power series

Trying square

In [ ]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_3C = Beta('ASC_CYCLE_3C', 0, None, None, 0)
ASC_PT_3C    = Beta('ASC_PT_3C', 0, None, None, 0)
ASC_CAR_3C   = Beta('ASC_CAR_3C', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_3C  = Beta('B_TIME_WALK_3C', 0, None, None, 0)
B_TIME_CYCLE_3C = Beta('B_TIME_CYCLE_3C', 0, None, None, 0)
B_TIME_PT_3C    = Beta('B_TIME_PT_3C', 0, None, None, 0)
B_TIME_CAR_3C   = Beta('B_TIME_CAR_3C', 0, None, None, 0)

# Power parameters
B_TIME_WALK_3C_squared  = Beta('B_TIME_WALK_3C_squared', 0, None, None, 0)
B_TIME_CYCLE_3C_squared = Beta('B_TIME_CYCLE_3C_squared', 0, None, None, 0)
B_TIME_PT_3C_squared    = Beta('B_TIME_PT_3C_squared', 0, None, None, 0)
B_TIME_CAR_3C_squared   = Beta('B_TIME_CAR_3C_squared', 0, None, None, 0)

# Generic cost coefficient
B_COST_3C = Beta('B_COST_3C', 0, None, None, 0)

age = Variable('age')
# B_AGE_CYCLE_3C = Beta('B_AGE_CYCLE_3C', 0, None, None, 0)
B_AGE_PT_3C = Beta('B_AGE_PT_3C', 0, None, None, 0)
B_AGE_CAR_3C = Beta('B_AGE_CAR_3C', 0, None, None, 0)

In [ ]:
V_walk_3C_piecewise_squared = (
    B_TIME_WALK_3C * dur_walking
    + B_TIME_WALK_3C_squared * dur_walking * dur_walking
)

V_cycle_3C_piecewise_squared = (
    ASC_CYCLE_3C
    # + B_AGE_CYCLE_3C * age
    + B_TIME_CYCLE_3C * dur_cycling
    + B_TIME_CYCLE_3C_squared * dur_cycling * dur_cycling
)

V_pt_3C_piecewise_squared = (
    ASC_PT_3C
    + B_AGE_PT_3C * age
    + B_TIME_PT_3C * dur_pt
    + B_TIME_PT_3C_squared * dur_pt * dur_pt
    + B_COST_3C * cost_transit
)

V_car_3C_piecewise_squared = (
    ASC_CAR_3C
    + B_AGE_CAR_3C * age
    + B_TIME_CAR_3C * dur_driving
    + B_TIME_CAR_3C_squared * dur_driving * dur_driving
    + B_COST_3C * cost_driving
)

V_3C_piecewise_squared ={1: V_walk_3C_piecewise_squared, 2: V_cycle_3C_piecewise_squared, 3: V_pt_3C_piecewise_squared, 4: V_car_3C_piecewise_squared}
logprob_3C_piecewise_squared = loglogit(V_3C_piecewise_squared, None, travel_mode)
biogeme_3C_piecewise_squared = bio.BIOGEME(database, logprob_3C_piecewise_squared)
biogeme_3C_piecewise_squared.modelName = 'model_3C_power_series_2'

results_3C_piecewise_squared = biogeme_3C_piecewise_squared.estimate()
print(results_3C_piecewise_squared.print_general_statistics())

Trying cubed

In [ ]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_3C = Beta('ASC_CYCLE_3C', 0, None, None, 0)
ASC_PT_3C    = Beta('ASC_PT_3C', 0, None, None, 0)
ASC_CAR_3C   = Beta('ASC_CAR_3C', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_3C  = Beta('B_TIME_WALK_3C', 0, None, None, 0)
B_TIME_CYCLE_3C = Beta('B_TIME_CYCLE_3C', 0, None, None, 0)
B_TIME_PT_3C    = Beta('B_TIME_PT_3C', 0, None, None, 0)
B_TIME_CAR_3C   = Beta('B_TIME_CAR_3C', 0, None, None, 0)

# Power parameters
B_TIME_WALK_3C_cubed  = Beta('B_TIME_WALK_3C_cubed', 0, None, None, 0)
B_TIME_CYCLE_3C_cubed = Beta('B_TIME_CYCLE_3C_cubed', 0, None, None, 0)
B_TIME_PT_3C_cubed    = Beta('B_TIME_PT_3C_cubed', 0, None, None, 0)
B_TIME_CAR_3C_cubed   = Beta('B_TIME_CAR_3C_cubed', 0, None, None, 0)

# Generic cost coefficient
B_COST_3C = Beta('B_COST_3C', 0, None, None, 0)

age = Variable('age')
# B_AGE_CYCLE_3C = Beta('B_AGE_CYCLE_3C', 0, None, None, 0)
B_AGE_PT_3C = Beta('B_AGE_PT_3C', 0, None, None, 0)
B_AGE_CAR_3C = Beta('B_AGE_CAR_3C', 0, None, None, 0)

In [ ]:
V_walk_3C_piecewise_cubed = (
    B_TIME_WALK_3C * dur_walking
    + B_TIME_WALK_3C_cubed * dur_walking * dur_walking * dur_walking
)

V_cycle_3C_piecewise_cubed = (
    ASC_CYCLE_3C
    # + B_AGE_CYCLE_3C * age
    + B_TIME_CYCLE_3C * dur_cycling
    + B_TIME_CYCLE_3C_cubed * dur_cycling * dur_cycling * dur_cycling
)

V_pt_3C_piecewise_cubed = (
    ASC_PT_3C
    + B_AGE_PT_3C * age
    + B_TIME_PT_3C * dur_pt
    + B_TIME_PT_3C_cubed * dur_pt * dur_pt * dur_pt
    + B_COST_3C * cost_transit
)

V_car_3C_piecewise_cubed = (
    ASC_CAR_3C
    + B_AGE_CAR_3C * age
    + B_TIME_CAR_3C * dur_driving
    + B_TIME_CAR_3C_cubed * dur_driving * dur_driving * dur_driving
    + B_COST_3C * cost_driving
)

V_3C_piecewise_cubed ={1: V_walk_3C_piecewise_cubed, 2: V_cycle_3C_piecewise_cubed, 3: V_pt_3C_piecewise_cubed, 4: V_car_3C_piecewise_cubed}
logprob_3C_piecewise_cubed = loglogit(V_3C_piecewise_cubed, None, travel_mode)
biogeme_3C_piecewise_cubed = bio.BIOGEME(database, logprob_3C_piecewise_cubed)
biogeme_3C_piecewise_cubed.modelName = 'model_3C_power_series_3'

results_3C_piecewise_cubed = biogeme_3C_piecewise_cubed.estimate()
print(results_3C_piecewise_cubed.print_general_statistics())

In [ ]:
V_pref = V_3B_unrestricted
results_pref = results_3B_unrestricted

## Model 4

In [ ]:
mu_motorized = Beta('mu_motorized', 1, 0, None, 0)
motorized = OneNestForNestedLogit(nest_param=mu_motorized, list_of_alternatives=[3, 4], name='motorized')
walk = OneNestForNestedLogit(nest_param=1, list_of_alternatives=[1], name='walk')
bike = OneNestForNestedLogit(nest_param=1, list_of_alternatives=[2], name='bike')
nests = NestsForNestedLogit(choice_set=V_pref, tuple_of_nests=(motorized, walk, bike))

In [ ]:
logprob_4 = lognested(V_pref, None, nests, travel_mode)
biogeme_4 = bio.BIOGEME(database, logprob_4)
biogeme_4.modelName = 'model_4'
results_4 = biogeme_4.estimate()
print(results_4.print_general_statistics())

In [ ]:
table_4 = results_4.get_estimated_parameters()
display(table_4)


The nested parameter is greater than one, consistently with the theory.

In [ ]:
lr_test_existing = results_4.likelihood_ratio_test(
    results_pref, significance_level=0.05
)
print(f'{lr_test_existing.statistic=:.3g}')
print(f'{lr_test_existing.threshold=:.3g}')
print(lr_test_existing.message)


## Market shares

In [ ]:
def get_weights(df):
    # Size of each segment in the population (N_g)
    census = {
        'female_44_less': 2841376,
        'female_45_more': 1519948,
        'male_44_less': 2926408,
        'male_45_more': 1379198,
    }

    # Size of the population (N)
    population_size = sum(census.values())

    # Identify each segment in the database
    filters = {
        'female_44_less': (df.age <= 44) & (df.female == 1),
        'female_45_more': (df.age >= 45) & (df.female == 1),
        'male_44_less': (df.age <= 44) & (df.female == 0),
        'male_45_more': (df.age >= 45) & (df.female == 0),
    }

    # Size of each segment in the sample (S_g)
    sample_segments = {
        segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
    }

    # Size of the sample (S)
    sample_size = sum(sample_segments.values())

    # Weights (omega_g = (N_g / N) * (S / S_g))
    weights = {
        segment_name: census[segment_name] * sample_size / (segment_size * population_size)
        for segment_name, segment_size in sample_segments.items()
    }

    return weights

def get_segments_size_in_sample(df):
    # Identify each segment in the database
    filters = {
        'female_44_less': (df.age <= 44) & (df.female == 1),
        'female_45_more': (df.age >= 45) & (df.female == 1),
        'male_44_less': (df.age <= 44) & (df.female == 0),
        'male_45_more': (df.age >= 45) & (df.female == 0),
    }

    # Size of each segment in the sample (S_g)
    sample_segments = {
        segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
    }

    return sample_segments


def get_market_shares(df, V, alpha = 0.9):
    """
    Input parameters :
    -> database : a panda dataframe,
    -> V : a dictionnary with the utility functions
       (1:walk, 2:cycle, 3:pt, 4:car)
    -> alpha : size of the confidence intervals
    Output :
    -> market_shares : dictionnary with the market shares 
        (1:walk, 2:cycle, 3:pt, 4:car)
    -> left : dict with the left bound of IC
    -> right : dict with the right bound of IC
    The three dicts are in %
    """
    weights = get_weights(df)

    filters = {
        'female_44_less': (df.age <= 44) & (df.female == 1),
        'female_45_more': (df.age >= 45) & (df.female == 1),
        'male_44_less': (df.age <= 44) & (df.female == 0),
        'male_45_more': (df.age >= 45) & (df.female == 0),
    }

    # Create a new dataframe to add the market shares in it
    df_ms = df.copy()

    # Add the weights in the dataframe
    for segment_name, segment_rows in filters.items():
        df_ms.loc[segment_rows, 'weight'] = weights[segment_name]

    # Useful variables for later
    weight = Variable('weight')
    travel_mode = Variable('travel_mode')

    # Apply model to the new dataframe
    database_ms = db.Database('lpcm_ms', df_ms)
    logprob_ms  = loglogit(V, None, travel_mode)
    biogeme_ms  = bio.BIOGEME(database_ms, logprob_ms)
    biogeme_ms.modelName = 'market_shares'
    results_ms  = biogeme_ms.estimate()

    # Normalization for probabilities
    normV = exp(V[1]) + exp(V[2]) + exp(V[3]) + exp(V[4])

    # Probabilities
    prob_walk  = exp(V[1]) / normV
    prob_cycle = exp(V[2]) / normV
    prob_pt    = exp(V[3]) / normV
    prob_car   = exp(V[4]) / normV

    # Computation of interesting formulas
    simulate = {
        'weight': weight,
        'Prob. walk': prob_walk,
        'Prob. cycle': prob_cycle,
        'Prob. PT': prob_pt,
        'Prob. car': prob_car
    }
    biosim_ms = bio.BIOGEME(database_ms, simulate)
    simulated_values = biosim_ms.simulate(results_ms.get_beta_values())

    simulated_values['Weighted walk'] = (
        simulated_values['weight'] * simulated_values['Prob. walk']
    )
    simulated_values['Weighted cycle'] = (
        simulated_values['weight'] * simulated_values['Prob. cycle']
    )
    simulated_values['Weighted PT'] = (
        simulated_values['weight'] * simulated_values['Prob. PT']
    )
    simulated_values['Weighted car'] = (
        simulated_values['weight'] * simulated_values['Prob. car']
    )

    # Market shares
    market_share_walk  = simulated_values['Weighted walk'].mean()
    market_share_cycle = simulated_values['Weighted cycle'].mean()
    market_share_pt    = simulated_values['Weighted PT'].mean()
    market_share_car   = simulated_values['Weighted car'].mean()

    market_shares = {
        1: market_share_walk*100, 
        2: market_share_cycle*100,
        3: market_share_pt*100,
        4: market_share_car*100,
    }

    # Confidence intervals
    biogeme_ms.bootstrap_samples = 100
    results_bootstrapping = biogeme_ms.estimate(run_bootstrap=True)
    betas = biogeme_ms.free_beta_names
    b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
    left, right = biosim_ms.confidence_intervals(b, alpha)

    left['Weighted walk']  = left['weight'] * left['Prob. walk']
    left['Weighted cycle'] = left['weight'] * left['Prob. cycle']
    left['Weighted PT']    = left['weight'] * left['Prob. PT']
    left['Weighted car']   = left['weight'] * left['Prob. car']

    right['Weighted walk']  = right['weight'] * right['Prob. walk']
    right['Weighted cycle'] = right['weight'] * right['Prob. cycle']
    right['Weighted PT']    = right['weight'] * right['Prob. PT']
    right['Weighted car']   = right['weight'] * right['Prob. car']

    left_market_share_walk  = left['Weighted walk'].mean()
    left_market_share_cycle = left['Weighted cycle'].mean()
    left_market_share_pt    = left['Weighted PT'].mean()
    left_market_share_car   = left['Weighted car'].mean()

    right_market_share_walk  = right['Weighted walk'].mean()
    right_market_share_cycle = right['Weighted cycle'].mean()
    right_market_share_pt    = right['Weighted PT'].mean()
    right_market_share_car   = right['Weighted car'].mean()

    left = {
        1: left_market_share_walk*100,
        2: left_market_share_cycle*100,
        3: left_market_share_pt*100,
        4: left_market_share_car*100,
    }

    right = {
        1: right_market_share_walk*100,
        2: right_market_share_cycle*100,
        3: right_market_share_pt*100,
        4: right_market_share_car*100,
    }

    return market_shares, left, right

def display_market_shares(df, V, alpha = 0.9):
    market_shares, left, right = get_market_shares(df, V, alpha)
    alternative_names = {
        1: 'walk',
        2: 'cycle',
        3: 'PT',
        4: 'car'
    }
    for i in np.arange(1,5):
        print(
            f'Market share for {alternative_names[i]}: {market_shares[i]:.2f}% '
            f'CI: ['
            f'{left[i]:.2f}%-'
            f'{right[i]:.2f}'
            f'%]'
        )

def get_sample_market_shares(df):
    market_shares = {}
    # Compute the segments size in the sample
    segments = {
        'walk': sum(df.travel_mode == 1),
        'cycle' : sum(df.travel_mode == 2),
        'pt' : sum(df.travel_mode == 3),
        'car' : sum(df.travel_mode == 4)
    }
    S = sum(segments.values())
    for name, value in segments.items():
        market_shares[name] = value/S
    return market_shares

def display_sample_market_shares(df):
    market_shares = get_sample_market_shares(df)
    for name, value in market_shares.items():
        print(f'Real market share within sample for {name}: {100*value:.2f}% ')

def get_weighted_market_shares(df):
    # The predicted share for alternative i is given by W(i) = 1/S * sum_{over individuals}(omega_n * P(i,x_n))
    # Where S the number of individuals in the sample, omega the weights, and P(i,x_n) the probability
    # that individual n choses alternative i. In our case we have two possibilities.
    # 1) We can compute the market shares within the sample, without taking the weights into account (equivalently,
    #    considering all weights equal to 1, the sampling is perfectly representative of the population)
    #    (that is what the function get_sample_market_shares does)
    # 2) We can take those weights into accounts, and using the formula above where P(i,x_n) is the percentage
    #    of individuals within the segment to which individual n belongs that chose alternative i. After some manipulations
    #    of equations, we find W(i) = sum_{over segments} (N_g / N) (S_g(i) / S_g)
    #    where N_g/N is the true proportion of the segment in the population, S_g the size of the segment in the sample
    #    and S_g(i) the number of persons within this segment having chosen alternative i
    census = {
        'female_44_less': 2841376,
        'female_45_more': 1519948,
        'male_44_less': 2926408,
        'male_45_more': 1379198,
    }
    N = sum(census.values())

    filters = {
        'female_44_less': sum((df.age <= 44) & (df.female == 1)),
        'female_45_more': sum((df.age >= 45) & (df.female == 1)),
        'male_44_less': sum((df.age <= 44) & (df.female == 0)),
        'male_45_more': sum((df.age >= 45) & (df.female == 0)),
    }
    S = sum(filters.values())

    market_shares = {}

    alternative_names = {1: 'walk', 2: 'cycle', 3: 'pt', 4: 'car'}

    for i in np.arange(1,5):
        filters_alternative = {
            'female_44_less': sum((df.age <= 44) & (df.female == 1) & (df.travel_mode == i)),
            'female_45_more': sum((df.age >= 45) & (df.female == 1) & (df.travel_mode == i)),
            'male_44_less': sum((df.age <= 44) & (df.female == 0) & (df.travel_mode == i)),
            'male_45_more': sum((df.age >= 45) & (df.female == 0) & (df.travel_mode == i)),
        }
        ms = 0
        for name in census.keys():
            ms += (census[name] / N) * (filters_alternative[name] / filters[name])
        market_shares[alternative_names[i]] = ms
    
    return market_shares

    

def display_weighted_market_shares(df):
    market_shares = get_weighted_market_shares(df)
    for name, value in market_shares.items():
        print(f'Weighted market share for {name}: {100*value:.2f}% ')

In [ ]:
#########################################
# Display of relevant data for the report
#########################################

# Size of segments within sample
segments_size = get_segments_size_in_sample(df)
print('Size of segments within sample : ', segments_size)
print('S = ', sum(segments_size.values()))

# Size of segments in total population
census = {
        'female_44_less': 2841376,
        'female_45_more': 1519948,
        'male_44_less': 2926408,
        'male_45_more': 1379198,
    }
print('Size of segments within population : ', census)
print('N = ', sum(census.values()))

# Weights
print(get_weights(df))

# Sample and weighted market shares
display_sample_market_shares(df)
display_weighted_market_shares(df)

# Market shares for a few models
print('Model 1A')
display_market_shares(df, V_1A)
print('Model 2B')
display_market_shares(df, V_2A)
print('Model 3B unrestricted')
display_market_shares(df, V_3B_unrestricted)
# Add model 4 once it is implemented
# print('Model 4')
# display_market_shares(df, V_4)

## Forecasting